# City of Toronto Ward Profile Demogrpahics Data

The City of Toronto's Ward Profiles, available on the Open Data Portal, provide detailed demographic data for each of the city's wards, offering insights into various aspects of the population. These profiles are based on custom tabulations from the 2021 2011 Census of Population conducted by Statistics Canada, with the data derived from the long-form Census questionnaire. 

## Setup Notebook

In [71]:
# Import 3rd party libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt

# Configure Notebook
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set_context("notebook")
import warnings
warnings.filterwarnings('ignore')

## Import City of Toronto Ward Profiles Demographic Excel File

In [73]:
# Read excel file
demographic_df = pd.read_excel('2023-WardProfiles-2011-2021-CensusData.xlsx', header = None)
# Set Column Names
demographic_df.columns = ['Grouping',
                         'Toronto',
                         'Ward 1',
                         'Ward 2',
                         'Ward 3',
                         'Ward 4',
                         'Ward 5',
                         'Ward 6',
                         'Ward 7',
                         'Ward 8',
                         'Ward 9',
                         'Ward 10',
                         'Ward 11',
                         'Ward 12',
                         'Ward 13',
                         'Ward 14',
                         'Ward 15',
                         'Ward 16',
                         'Ward 17',
                         'Ward 18',
                         'Ward 19',
                         'Ward 20',
                         'Ward 21',
                         'Ward 22',
                         'Ward 23',
                         'Ward 24',
                         'Ward 25']


# View DataFrame
demographic_df.head(50)

,Grouping,Toronto,Ward 1,Ward 2,Ward 3,Ward 4,Ward 5,Ward 6,Ward 7,Ward 8,...,Ward 16,Ward 17,Ward 18,Ward 19,Ward 20,Ward 21,Ward 22,Ward 23,Ward 24,Ward 25
0,City of Toronto Profiles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,City of Toronto: City Wards,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021 Census,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Source:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"Statistics Canada, 2021 Census, Custom Tabulat...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Wards:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,25 Ward model,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Clean Demographic Data

The structure of this file will make it challenging to parse. This file contains the ward numbers as the column headers and data entries in the rows. The file is divided into categories at random row intervals. This is denoted by inserting a row of cells with their fill colour set to gray that contains the name of the new section (see Figure 1). Each category is divided into multiple sub-categories at random intervals that are denoted by either inserting one or two blank rows (see Figure 2). This presents a challenge to extract specific categories of data because there does not exist a unique entry in the category heading that distinguishes it from data entries. 

See this website for the full dataset https://open.toronto.ca/dataset/ward-profiles-25-ward-model/

### Figure 1
![Figure 1](Figure1.png)

### Figure 2
![Figure 2](Figure2.png)

In [78]:
# Determine indices of shaded rows as these are the category headers of the census data (eg. population, dwellings etc.), store in a list
shaded_rows = [17, 42, 74, 92, 136, 995, 1014, 1030, 1301, 1358] # counted manually in excel worksheet

In [79]:
# Initialize list to store the data frames of each demographic category (eg. population, dwellings etc.)
category_dfs = []

# Loop through the list of indices of category headers
for i in range(len(shaded_rows)):
    
    # Initialize list to store the data frames of each subcategory 
    subcategory_dfs = []
    
    # From first category to second-last category (prevents indexing out of range)
    if shaded_rows[i] < shaded_rows[-1]:
        category_name = demographic_df.iloc[shaded_rows[i]-1,0] # Get the name of the category
        
        # Slice the rows between the current category header and the next category header
        category_df = demographic_df.iloc[shaded_rows[i] : shaded_rows[i+1] - 1].dropna(how='all') 
        category_df['Category'] = category_name # Creates a new column that contains the category of the row
        category_df = category_df.reset_index(drop=True) 
        
        # Subcategories are distinguished by containing 'Total' in grouping description, check which rows are subcategory headers
        subcategory_rows = category_df['Grouping'].notnull() & category_df['Grouping'].str.contains('Total')
        
        # Get the indices of subcategory rows
        subcategory_indices = category_df[subcategory_rows].index.tolist()
        
        # Loop through the list of indices of subcategory headers
        for j in range(len(subcategory_indices)):
            
            # From first category to second-last category (prevents indexing out of range)
            if subcategory_indices[j] < subcategory_indices[-1]:
                subcategory_name = category_df.iloc[subcategory_indices[j],0] # Get the name of the subcategory
                
                #Slice the rows between the current subcategory header and the next subcategory header
                subcategory_df = category_df.iloc[subcategory_indices[j] : subcategory_indices[j+1] - 1]
                subcategory_df['Subcategory'] = subcategory_name # Creates a new column that contains the subcategory of the row
                subcategory_dfs.append(subcategory_df) # Append the data frame for this subcategory to the list containing the other subcategory data frames
            
            # The last subcategory is handled differently
            else:
                subcategory_name = category_df.iloc[subcategory_indices[j],0] # Get the name of the subcategory
                
                # Slice the rows from the current subcategory header to the end of the category's data frame
                subcategory_df = category_df.iloc[subcategory_indices[j]:]
                subcategory_df['Subcategory'] = subcategory_name # Creates a new column that contains the subcategory of the row
                subcategory_dfs.append(subcategory_df)  # Append the data frame for this subcategory to the list containing the other subcategory data frames

        category_df = pd.concat(subcategory_dfs) # combine all the subcategory data frames together to create the category data frame
        category_dfs.append(category_df) # append the current category data frame to the list containing the other category data frames
        
    # For the last category in the sheet
    else:
        category_name = demographic_df.iloc[shaded_rows[i]-1,0] # Get the name of the category
        
        #Slice the rows between the current category header and the end of the sheet (as this is the last category header)
        category_df = demographic_df.iloc[shaded_rows[i]:].dropna(how='all')
        category_df['Category'] = category_name # Creates a new column that contains the category of the row
        category_dfs.append(category_df)
        category_df = category_df.reset_index(drop=True)

        # Subcategories are distinguished by containing 'Total' in grouping description, check which rows are subcategory headers
        subcategory_rows = category_df['Grouping'].notnull() & category_df['Grouping'].str.contains('Total')
        
        # Get the indices of subcategory rows
        subcategory_indices = category_df[subcategory_rows].index.tolist()

        # Loop through the list of indices of subcategory headers
        for j in range(len(subcategory_indices)):
            
            # From first category to second-last category (prevents indexing out of range)
            if subcategory_indices[j] < subcategory_indices[-1]:
                subcategory_name = category_df.iloc[subcategory_indices[j],0] # Get the name of the subcategory
                
                 #Slice the rows between the current subcategory header and the next subcategory header
                subcategory_df = category_df.iloc[subcategory_indices[j] : subcategory_indices[j+1] - 1]
                subcategory_df['Subcategory'] = subcategory_name # Creates a new column that contains the subcategory of the row
                subcategory_dfs.append(subcategory_df) # Append the data frame for this subcategory to the list containing the other subcategory data frames
            
            # The last subcategory is handled differently
            else:
                subcategory_name = category_df.iloc[subcategory_indices[j],0] # Get the name of the subcategory
                
                # Slice the rows from the current subcategory header to the end of the category's data frame
                subcategory_df = category_df.iloc[subcategory_indices[j]:]
                subcategory_df['Subcategory'] = subcategory_name # Creates a new column that contains the subcategory of the row
                subcategory_dfs.append(subcategory_df) # Append the data frame for this subcategory to the list containing the other subcategory data frames

        category_df = pd.concat(subcategory_dfs) # combine all the subcategory data frames together to create the category data frame
        category_dfs.append(category_df) # append the current category data frame to the list containing the other category data frames


combined_df = pd.concat(category_dfs) # Combine all category data frames together to create the entire demographic profile data frame
combined_df.set_index(['Category', 'Subcategory', 'Grouping'], inplace=True) # Create a 3-level multi-index

# View DataFrame
display(combined_df)

Toronto  \
Category                 Subcategory    Grouping                                                      
Population               Total - Age    Total - Age                                         2761285   
                                          0 to 4 years                                       123385   
                                          5 to 9 years                                       128805   
                                          10 to 14 years                                     131150   
                                          15 to 19 years                                     133820   
...                                                                                             ...   
Income and Shelter Costs Total income %     Canada Pension Plan (CPP) and QuÚbec Pensio...      3.5   
                                            Old Age Security pension (OAS) and Guarante...      2.9   
                                            Employment Insurance (EI) benefits %                1.3   
                                            Child benefits $                                    2.8   
                                            Other government transfers %                        9.5   

                                                                                            Ward 1  \
Category                 Subcategory    Grouping                                                     
Population               Total - Age    Total - Age                                         115120   
                                          0 to 4 years                                        5755   
                                          5 to 9 years                                        6160   
                                          10 to 14 years                                      6585   
                                          15 to 19 years                                      7085   
...                                                                                            ...   
Income and Shelter Costs Total income %     Canada Pension Plan (CPP) and QuÚbec Pensio...     5.2   
                                            Old Age Security pension (OAS) and Guarante...     3.8   
                                            Employment Insurance (EI) benefits %               2.1   
                                            Child benefits $                                   6.9   
                                            Other government transfers %                      16.8   

                                                                                            Ward 2  \
Category                 Subcategory    Grouping                                                     
Population               Total - Age    Total - Age                                         117200   
                                          0 to 4 years                                        5105   
                                          5 to 9 years                                        5825   
                                          10 to 14 years                                      6370   
                                          15 to 19 years                                      6540   
...                                                                                            ...   
Income and Shelter Costs Total income %     Canada Pension Plan (CPP) and QuÚbec Pensio...     3.7   
                                            Old Age Security pension (OAS) and Guarante...     4.2   
                                            Employment Insurance (EI) benefits %               1.1   
                                            Child benefits $                                   2.2   
                                            Other government transfers %                       7.5   

                                                                                            Ward 3

In [80]:
combined_df = combined_df.reset_index() # Reset index for easier filtering

#View DataFrame
display(combined_df)

,Category,Subcategory,Grouping,Toronto,Ward 1,Ward 2,Ward 3,Ward 4,Ward 5,Ward 6,...,Ward 16,Ward 17,Ward 18,Ward 19,Ward 20,Ward 21,Ward 22,Ward 23,Ward 24,Ward 25
0,Population,Total - Age,Total - Age,2761285,115120,117200,139920,104715,115675,107355,...,94335,112590,117130,108500,110095,111560,103690,94025,102755,101485
1,Population,Total - Age,0 to 4 years,123385,5755,5105,6765,4770,6045,5265,...,4600,4375,4525,6035,5610,5165,4000,3770,5195,4365
2,Population,Total - Age,5 to 9 years,128805,6160,5825,6180,5015,6060,5205,...,4755,4780,4480,6455,6230,5880,4480,4660,5890,5080
3,Population,Total - Age,10 to 14 years,131150,6585,6370,5515,5230,6360,5085,...,5225,5010,4425,6195,6285,5920,4815,4665,6090,5590
4,Population,Total - Age,15 to 19 years,133820,7085,6540,5160,4680,6390,5705,...,5225,5090,5035,5480,6425,6440,4930,4750,6030,6105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1477,Income and Shelter Costs,Total income %,Canada Pension Plan (CPP) and QuÚbec Pensi...,3.5,5.2,3.7,2.8,2.5,5.1,4.5,...,4.7,4.3,3.6,2.8,4.2,5.3,7.2,7.1,5.5,4.9
1478,Income and Shelter Costs,Total income %,Old Age Security pension (OAS) and Guarant...,2.9,3.8,4.2,3,2.3,4,3.4,...,4,3.5,2.6,2.8,3.5,4.1,4.3,3.5,3.9,4.2
1479,Income and Shelter Costs,Total income %,Employment Insurance (EI) benefits %,1.3,2.1,1.1,1.3,1.2,2.2,1.8,...,1.4,1.3,1.1,1.3,1.8,1.9,1.7,1.9,1.8,1.7
1480,Income and Shelter Costs,Total income %,Child benefits $,2.8,6.9,2.2,1.6,1.4,5.4,3.7,...,4.2,2.8,2.1,2.4,5,5.7,4.3,5.8,6.6,4.2


In [81]:
# Export clean DataFrame to csv for clustering
combined_df.to_csv('../../Clean Data/demographic_data_clean.csv', index=False)

## EDA

Some EDA was conducted to initially understand, but it is not very significant and thus we decided to comment it out. The code is left in for your reference.

### Age in Ward

In [85]:
# age_df = combined_df[combined_df['Subcategory'] == 'Total - Age']
# age_df = age_df.transpose()
# age_df.columns = age_df.loc['Grouping']
# age_df.drop(['Category', 'Subcategory', 'Grouping'], axis = 0, inplace = True)
# age_df = age_df.apply(lambda row: (row / row[0])*100, axis=1)
# age_df = age_df.drop(columns = ['Total - Age', 'Median age'], axis = 1)
# age_df

In [86]:
# for ward in age_df.index:
#     # Extract data from each row
#     age_data = age_df.loc[ward]

#     # Plot the age distribution
#     plt.figure(figsize=(8, 4)) 
#     sns.barplot(data = age_data)

#     plt.title(ward)
#     plt.xlabel('Age')
#     plt.ylabel("Percentage of Ward's Population")
#     plt.xticks(rotation=90)
#     plt.show

### Dwelling Age in Ward

In [88]:
# dwelling_df = combined_df[combined_df['Subcategory'] == 'Total - Occupied private dwellings by period of construction - 25% sample data']
# dwelling_df = dwelling_df.transpose()
# dwelling_df.columns = dwelling_df.loc['Grouping']
# dwelling_df.drop(['Category', 'Subcategory', 'Grouping'], axis = 0, inplace = True)
# dwelling_df = dwelling_df.apply(lambda row: (row / row[0])*100, axis=1)
# dwelling_df = dwelling_df.drop(columns = ['Total - Occupied private dwellings by period of construction - 25% sample data'], axis = 1)
# dwelling_df

In [89]:
# for ward in dwelling_df.index:
#     # Extract data from each row
#     dwelling_age_data = dwelling_df.loc[ward]

#     # Plot the age distribution
#     plt.figure(figsize=(8, 4)) 
#     sns.barplot(data = dwelling_age_data)

#     plt.title(ward)
#     plt.xlabel('Age')
#     plt.ylabel("Percentage of Dwellings in Ward")
#     plt.xticks(rotation=90)
#     plt.show

### Migration, Mobllity & Languages in Ward

In [91]:
# mml_df = combined_df[combined_df['Category'] == 'Migration, Mobllity & Languages']
# mml_df

Explore Generation Status

In [93]:
# generation_status_df =  mml_df[mml_df['Subcategory'] == 'Total - Generation status for the population in private households - 25% sample data']
# generation_status_df = generation_status_df.transpose()
# generation_status_df.columns = generation_status_df.loc['Grouping']
# generation_status_df.drop(['Category', 'Subcategory', 'Grouping'], axis = 0, inplace = True)
# generation_status_df = generation_status_df.apply(lambda row: (row / row[0])*100, axis=1)
# generation_status_df = generation_status_df.drop(columns = ['Total - Generation status for the population in private households - 25% sample data'], axis = 1)
# generation_status_df

In [94]:
# for ward in generation_status_df.index:
#     # Extract data from each row
#     generation_status_data = generation_status_df.loc[ward]

#     # Plot the age distribution
#     plt.figure(figsize=(6, 3)) 
#     sns.barplot(data = generation_status_data)

#     plt.title(ward)
#     plt.xlabel('Generation Status')
#     plt.ylabel("Percentage of Residents in Ward")
#     plt.xticks(rotation=90)
#     plt.show

Explore Places of Birth of Immigrant Population

In [96]:
# birth_location_df = mml_df[mml_df['Subcategory'] == 'Total - Selected places of birth for the immigrant population in private households - 25% sample data']
# birth_location_df['Grouping'] = birth_location_df['Grouping'].str.strip()
# birth_location_df = birth_location_df.transpose()
# birth_location_df.columns = birth_location_df.loc['Grouping']
# birth_location_df.drop(['Category', 'Subcategory', 'Grouping'], axis = 0, inplace = True)
# birth_location_df = birth_location_df.apply(lambda row: (row / row[0])*100, axis=1)
# birth_location_df = birth_location_df[['Americas', 'Europe',  'Africa', 'Asia']]
# birth_location_df

In [97]:
# for ward in birth_location_df.index:
#     # Extract data from each row
#     birth_location_data = birth_location_df.loc[ward]

#     # Plot the age distribution
#     plt.figure(figsize=(6, 3)) 
#     sns.barplot(data = birth_location_data)

#     plt.title(ward)
#     plt.xlabel('Birth Location')
#     plt.ylabel("Percentage of Residents in Ward")
#     plt.xticks(rotation=90)
#     plt.show

### Education Level in Ward

In [99]:
# education_df = combined_df[combined_df['Subcategory'] == 'Total - Highest certificate, diploma or degree for the population aged 15 years and over in private households - 25% sample data']
# education_df['Grouping'] = education_df['Grouping'].str.strip()
# education_df = education_df.transpose()
# education_df.columns = education_df.loc['Grouping']
# education_df.drop(['Category', 'Subcategory', 'Grouping'], axis = 0, inplace = True)
# education_df = education_df.apply(lambda row: (row / row[0])*100, axis=1)
# education_df["Bachelor’s degree or higher"] = education_df["Bachelor's degree"] + education_df["University certificate or diploma above bachelor level"]
# education_df["Master's, Doctorate, Medical Degree"] = education_df['Degree in medicine, dentistry, veterinary medicine or optometry'] + education_df["Master's degree"] + education_df['Earned doctorate']
# education_df = education_df[["No certificate, diploma or degree", "High (secondary) school diploma or equivalency certificate", 
#                              "Postsecondary certificate, diploma or degree", "Bachelor’s degree or higher", "Master's, Doctorate, Medical Degree"]]
# education_df

In [100]:
# for ward in education_df.index:
#     # Extract data from each row
#     education_data = education_df.loc[ward]

#     # Plot the age distribution
#     plt.figure(figsize=(6, 3)) 
#     sns.barplot(data = education_data)

#     plt.title(ward)
#     plt.xlabel('Education Level')
#     plt.ylabel("Percentage of Residents in Ward")
#     plt.xticks(rotation=90)
#     plt.show

### Place of Work in Ward

In [102]:
# labour_df = combined_df[combined_df['Subcategory'] == 'Total - Place of work status for the employed labour force aged 15 years and over in private households - 25% sample data']
# labour_df = labour_df.transpose()
# labour_df.columns = labour_df.loc['Grouping']
# labour_df.drop(['Category', 'Subcategory', 'Grouping'], axis = 0, inplace = True)
# labour_df = labour_df.apply(lambda row: (row / row[0])*100, axis=1)
# labour_df = labour_df.drop(columns = ['Total - Place of work status for the employed labour force aged 15 years and over in private households - 25% sample data', 
#                                      '  Worked outside Canada'], axis = 1)
# labour_df

In [103]:
# for ward in labour_df.index:
#     # Extract data from each row
#     labour_data = labour_df.loc[ward]

#     # Plot the age distribution
#     plt.figure(figsize=(6, 3)) 
#     sns.barplot(data = labour_data)

#     plt.title(ward)
#     plt.xlabel('Place of Work')
#     plt.ylabel("Percentage of Residents in Ward")
#     plt.xticks(rotation=90)
#     plt.show

### Income Breakdown in Ward

In [105]:
# average_income_df = combined_df[((combined_df['Grouping'] == '  Average total income of households in 2020 ($)') &
#                                 (combined_df['Subcategory'] == 'Total - Income statistics in 2020 for private households by household size - 25% sample data'))]
# average_income_df = average_income_df.transpose()
# average_income_df.columns = average_income_df.loc['Grouping']
# average_income_df.drop(['Category', 'Subcategory', 'Grouping'], axis = 0, inplace = True)
# average_income_df = average_income_df.sort_values(by = '  Average total income of households in 2020 ($)', ascending = True)
# average_income_df

In [106]:
# sns.barplot(x = average_income_df.index, y = average_income_df['  Average total income of households in 2020 ($)'])

# plt.title("Average Income By Ward")
# plt.xlabel("Ward")
# plt.xticks(rotation=90)
# plt.show()

In [107]:
# #https://www.springfinancial.ca/blog/lifestyle/middle-class-income-in-canada-by-province

# class_df = combined_df[combined_df['Subcategory'] == 'Total - Total income groups in 2020 for the population aged 15 years and over in private households - 25% sample data']
# class_df['Grouping'] = class_df['Grouping'].str.strip()
# class_df = class_df.transpose()
# class_df.columns = class_df.loc['Grouping']
# class_df.drop(['Category', 'Subcategory', 'Grouping'], axis = 0, inplace = True)

# class_df['Lower Class'] = (class_df['Under $10,000 (including loss)'] + class_df['$10,000 to $19,999'] + class_df['$20,000 to $29,999'] +
#                           class_df['$30,000 to $39,999'] + class_df['$40,000 to $49,999'])

# class_df['Middle Class'] = (class_df['$50,000 to $59,999'] + class_df['$60,000 to $69,999'] + class_df['$70,000 to $79,999'] + 
#                            class_df['$80,000 to $89,999'] + class_df['$90,000 to $99,999'])

# class_df['Upper Class'] = class_df['$100,000 and over']

# class_df = class_df.apply(lambda row: (row / row[0])*100, axis=1)
# class_df = class_df[['Lower Class', 'Middle Class', 'Upper Class']]
# class_df

In [108]:
# for ward in class_df.index:
#     # Extract data from each row
#     class_data = class_df.loc[ward]

#     # Plot the age distribution
#     plt.figure(figsize=(6, 3)) 
#     sns.barplot(data = class_data)

#     plt.title(ward)
#     plt.xlabel('Class')
#     plt.ylabel("Percentage of Ward's Population")
#     plt.xticks(rotation=90)
#     plt.show